**Conversion du modèle Pytorch en Onnx**

Importation des librairies

In [1]:
import torch
import torch.onnx
from model_scn2     import ExtendedSimpleCNN2D

Exécution du code

In [ ]:
# Importation du modèle utilisé
model = ExtendedSimpleCNN2D(3,8)

# Importation des poids
checkpoint = torch.load('./runs/class_langage-1/best_checkpoint.pth', weights_only=True)

# Ajout des poids sur le modèle
model.load_state_dict(checkpoint['model_state'])

# Mise en mode évaluation du modèle
model.eval()

# Exemple de donnée d'entrée : 1 image RGB (3 canneaux) de taille 128x128
dummy_input = torch.randn(1, 3, 128, 128)

# Exporter le modèle au format ONNX
torch.onnx.export(model, dummy_input, ".\models\onnx\model2.onnx", opset_version=11,input_names=['input'], output_names=['output'])

**Fin du code la suite ce fait sur le collab du prof car impossible sur Windows : https://colab.research.google.com/drive/1HLO2sIK_VDD7MO6CnNbWbp30GEquRWgk?usp=sharing**

<span style="color:red; font-family: 'Arial', sans-serif; font-weight: bold; font-size: 40px;">NON UTILISE | ANCIEN CODE EN ARCHIVE POUR GARDER DES TRACES<span>

Méthode Moi **Conversion Onnx -> Tensorflow**

Importation des libraires onnx nécéssaire pour la conversion en fichier tensorflow

In [ ]:
import onnx

from onnx_tf.backend import prepare

Récupération du model onnx

In [4]:
onnx_path = '.\onnx\model2.onnx'
onnx_model = onnx.load(onnx_path)

Conversion du modèle onnx en tensorflow

In [ ]:
tf_model = prepare(onnx_model)

tf_model.export_graph('./models/tensorflow_model')

Méthode du prof **Conversion onnx -> tensorflow**

Instalation des librairies nécéssaires

In [ ]:
!pip install onnx2tf onnx-graphsurgeon onnx onnxruntime sng4onnx

Conversion du modèle onnx en tensorflow

In [ ]:
!onnx2tf -i ./02_models/onnx/model2.onnx

**Conversion du modèle tensorflow en tensorflow-js**

In [ ]:
# Convert TF to TF-JS

!tensorflowjs_converter \
    --input_format=tf_saved_model \
    --saved_model_tags=serve \
    /saved_model \
    /saved_model_tfjs

**Test du modèle tensorflow**

In [18]:
import numpy as np
from PIL import Image

# Charger l'image et la préparer
print('Image 1:')
img = Image.open('two.jpg').resize((128, 128)).convert('RGB')  # Convertir en RGB pour 3 canaux

# Convertir l'image en un tableau numpy avec la forme correcte (1, 3, 128, 128)
image_array = np.asarray(img, dtype=np.float32)  # (128, 128, 3)
image_array = np.transpose(image_array, (2, 0, 1))  # Changer la forme à (3, 128, 128)
image_array = image_array[np.newaxis, :]  # Ajouter la dimension du batch, forme (1, 3, 128, 128)

# Afficher la forme de l'image pour débogage
print(f"Shape de l'entrée : {image_array.shape}")

# Utilisez le nom exact de l'entrée trouvé précédemment
input_name = 'input'  # Remplacez par le nom exact trouvé

# Assurez-vous que l'image est dans la forme correcte (1, 3, 128, 128)
output = tf_model.run(image_array)  # Le modèle attend un dictionnaire
print('The digit is classified as ', np.argmax(output))
print(output)

# Image 2
print('Image 2:')
img = Image.open('three.jpg').resize((128, 128)).convert('RGB')  # Convertir en RGB

# Convertir l'image en un tableau numpy avec la forme correcte (1, 3, 128, 128)
image_array = np.asarray(img, dtype=np.float32)  # (128, 128, 3)
image_array = np.transpose(image_array, (2, 0, 1))  # Changer la forme à (3, 128, 128)
image_array = image_array[np.newaxis, :]  # Ajouter la dimension du batch, forme (1, 3, 128, 128)

# Afficher la forme de l'image pour débogage
print(f"Shape de l'entrée : {image_array.shape}")

# Exécuter le modèle
output = tf_model.run(image_array)  # Remplacer 'tf_model' par votre modèle ONNX
print('The digit is classified as ', np.argmax(output))
print(output)


Image 1:
Shape de l'entrée : (1, 3, 128, 128)
The digit is classified as  4
Outputs(output=array([[-142.21492, -127.56512,  -51.24095, -164.81468,  -44.7522 ,
        -109.90031, -235.22833, -136.96535]], dtype=float32))
Image 2:
Shape de l'entrée : (1, 3, 128, 128)
The digit is classified as  2
Outputs(output=array([[-107.78237 , -138.40663 ,  -34.93393 , -154.97597 ,  -72.768364,
        -138.90031 , -336.4368  , -168.51442 ]], dtype=float32))
